In [1]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
import pickle
import numpy as np

for i in range(1,6):
#     path = 'cifar-10-python\cifar-10-batches-py\data_batch_' + str(i)
    with open(r'cifar-10-python\cifar-10-batches-py\data_batch_' + str(i), mode='rb') as file:
        # note the encoding type is 'latin1'
        batch = pickle.load(file, encoding='latin1')
    if i == 1:  
        train_X = (batch['data'].reshape((len(batch['data']), 3, 32, 32)).transpose(0, 2, 3, 1)).astype('float32')
        train_Y = batch['labels']
    else:
        x_train_temp = (batch['data'].reshape((len(batch['data']), 3, 32, 32)).transpose(0, 2, 3, 1)).astype('float32')
        y_train_temp = batch['labels']
        train_X = np.concatenate((train_X,x_train_temp),axis = 0)
        train_Y = np.concatenate((train_Y,y_train_temp),axis=0)

#path = 'cifar-10-python\cifar-10-batches-py\test_batch'
with open(r'cifar-10-python\cifar-10-batches-py\test_batch','rb') as file:
    # note the encoding type is 'latin1'
    batch = pickle.load(file, encoding='latin1')
    test_X = (batch['data'].reshape((len(batch['data']), 3, 32, 32)).transpose(0, 2, 3, 1)).astype('float32')
    test_Y = batch['labels']

In [12]:
train_X.shape

(50000, 32, 32, 3)

In [3]:
import matplotlib.pyplot as plt
import tensorflow as tf

In [4]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from tensorflow.keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras.layers import BatchNormalization
from keras import regularizers
from keras.layers import Activation
from tensorflow.keras.optimizers import Adam

In [5]:
train_x=train_X.astype('float32')
test_X=test_X.astype('float32')
 
train_X=train_X/255.0
test_X=test_X/255.0

In [6]:
train_Y=np_utils.to_categorical(train_Y)
test_Y=np_utils.to_categorical(test_Y)
 
num_classes=test_Y.shape[1]

In [ ]:
# l1 Regularization is that we have to penalize our weights by 
#adding absolute values of weight in our loss function

### kernel_regularizer='l1' on the layers

###

#2 Regularization is another regularization technique which is 
#also known as Ridge regularization. In L2 regularization we add 
#the squared magnitude of weights to penalize our lost function.

####kernel_regularizer='l2'  on the layers

####

###dropout The main idea behind using dropout is that we randomly 
#turn off some neurons in our layer based on some probability.

### Dropout(0.5) add a layer 

###

###batch normalization he main idea behind batch normalization is 
#that we normalize the input layer by using several techniques

### BatchNormalization() add a layer

In [ ]:
#### explanation

model=Sequential() #create empty sequential model and then add layers

#Layer 1: conv layer, filter size 3X3, stride size 1 in both dimensions,
#depth 32. Padding same and activation relu will apply to all layers.
#we will use ReLU activation for all our layers, except for the last layer
#remember that ReLU doesn't map negative values (no negative values here)
#no specification of stride default setting = 1
#input shape needs to be specified, but not for the following layers
model.add(Conv2D(32,(3,3), activation='relu', padding='same',\
                 input_shape=(32,32,3)))

#Layer 2: conv layer, filter size 3X3, stride size 1 in both dimensions,
#depth 32. Padding same and activation relu will apply to all layers.
#we would need padding 1 to achieve the same width and height, but 
#we will use 'same' padding for all the conv layers, aka zero pad.
model.add(Conv2D(32,(3,3), activation='relu', padding='same'))
    
#Layer 3: max pooling layer, pool size 2X2, stride 2 in both dimensions,
#max pooling layer stride default given by pool size
model.add(MaxPooling2D(pool_size=(2,2)))

#Layer 4: dropout layer with probability 25% of dropout, to prevent overfitting
model.add(Dropout(0.25))

#Layers 5-8: same but depth of conv layer is 64 instead of 32
model.add(Conv2D(64,(3,3), activation='relu', padding='same'))
model.add(Conv2D(64,(3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

#Layer 9: FC (Fully-Connected) layer. Now our neurons are not just in 
#on row, but spatially arranged in a cube-like format. We need to make
#them into one row, flattening. Flatten layer.
model.add(Flatten()) #now one row
model.add(Dense(512,activation='relu')) #dense (FC) layer. 

#Layer 10: another dropout of probability 50%
model.add(Dropout(0.5))

#Layers 11-12: another dense (FC) layer with 10 neurons and sofrmax activation
#last layer, softmax, only transforms the output of the previous layer 
#into probability distributions, which is the final goal
model.add(Dense(10,activation='softmax'))

In [7]:
## layers 1: original ==best one
model=Sequential()
model.add(Conv2D(32,(3,3),input_shape=(32,32,3),
    padding='same',activation='relu',
    kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Conv2D(32,(3,3),activation='relu',padding='same',kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(512,activation='relu',kernel_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [76]:
## layers 2: original + batch normalization == a lot overfitting
model=Sequential()
model.add(Conv2D(32,(3,3),input_shape=(32,32,3),
    padding='same',activation='relu',
    kernel_constraint=maxnorm(3)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),activation='relu',padding='same',kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(512,activation='relu',kernel_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))

In [11]:
## layers 3: batch normalization remove batch size == a lot overfitting



model=Sequential()

model.add(Conv2D(32,(3,3),input_shape=(32,32,3),
    padding='same',activation='relu',
    kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),activation='relu',padding='same',kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(512,activation='relu',kernel_constraint=maxnorm(3)))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(512,activation='relu',kernel_constraint=maxnorm(3)))
model.add(BatchNormalization())
model.add(Dense(512,activation='relu',kernel_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(512,activation='relu',kernel_constraint=maxnorm(3)))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(512,activation='relu',kernel_constraint=maxnorm(3)))
model.add(BatchNormalization())
model.add(Dense(512,activation='relu',kernel_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(512,activation='relu',kernel_constraint=maxnorm(3)))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(512,activation='relu',kernel_constraint=maxnorm(3)))
model.add(BatchNormalization())
model.add(Dense(512,activation='relu',kernel_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))


In [18]:
### layers 4:   epoch50 batch64 good? 


weight_decay = 1e-4

model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (3,3), padding='same',input_shape=(32,32,3), kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 32, kernel_size = (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.2))

model.add(Conv2D(filters = 64, kernel_size = (3,3), padding='same',input_shape=(32,32,3), kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 64, kernel_size = (3,3), padding='same', input_shape=(32,32,3),kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))

model.add(Conv2D(filters = 128, kernel_size = (3,3), padding='same',input_shape=(32,32,3), kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 128, kernel_size = (3,3), padding='same', input_shape=(32,32,3),kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.4))



model.add(Flatten())

model.add(Dense(units = 10, activation = 'softmax'))
    

In [15]:
### layers 5:  epoch100 batch32 == need testing

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(32,32,3)))
model.add(Activation('relu'))

model.add(Conv2D(48, (3, 3), padding='same',
                input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(80, (3, 3), padding='same',
                 input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same',
                input_shape=(32,32,3)))
model.add(MaxPooling2D())
model.add(Dropout(0.25))


model.add(Dense(500))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(num_classes,activation='relu'))
model.add(Activation('softmax'))

In [96]:
sgd=SGD(lr=0.1,momentum=0.9,decay=(0.01/25),nesterov=False)

model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])

In [19]:
adam=Adam(learning_rate=0.01,decay = 1e-6,amsgrad=False)

model.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])

In [20]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 activation_10 (Activation)  (None, 32, 32, 32)        0         
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_13 (Conv2D)          (None, 32, 32, 32)        9248      
                                                                 
 activation_11 (Activation)  (None, 32, 32, 32)        0         
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 32)       128       
 hNormalization)                                      

In [21]:
model.fit(train_X,train_Y,validation_data=(test_X,test_Y),epochs=15,batch_size=64)

Epoch 1/15
782/782 [==============================] - 155s 197ms/step - loss: 2.0910 - accuracy: 0.4226 - val_loss: 1.5674 - val_accuracy: 0.4973
Epoch 2/15
782/782 [==============================] - 155s 198ms/step - loss: 1.2795 - accuracy: 0.6180 - val_loss: 1.1842 - val_accuracy: 0.6717
Epoch 3/15
782/782 [==============================] - 160s 205ms/step - loss: 1.2327 - accuracy: 0.6707 - val_loss: 1.2749 - val_accuracy: 0.6701
Epoch 4/15
782/782 [==============================] - 167s 213ms/step - loss: 1.2403 - accuracy: 0.6869 - val_loss: 1.5648 - val_accuracy: 0.5852
Epoch 5/15
782/782 [==============================] - 165s 211ms/step - loss: 1.2585 - accuracy: 0.6932 - val_loss: 1.3551 - val_accuracy: 0.6559
Epoch 6/15
782/782 [==============================] - 158s 202ms/step - loss: 1.2540 - accuracy: 0.7003 - val_loss: 1.2404 - val_accuracy: 0.7023
Epoch 7/15
782/782 [==============================] - 156s 199ms/step - loss: 1.2361 - accuracy: 0.7041 - val_loss: 1.1981 -

In [22]:

loss8, acc8 =model.evaluate(train_X,train_Y)
print(f"Loss is {loss8},\nAccuracy is {acc8 * 100}")
loss8, acc8 = model.evaluate(test_X, test_Y)
print(f"Loss is {loss8},\nAccuracy is {acc8 * 100}")


1563/1563 [==============================] - 55s 35ms/step - loss: 1.2471 - accuracy: 0.7081
Loss is 1.2470686435699463,
Accuracy is 70.80600261688232
313/313 [==============================] - 11s 35ms/step - loss: 1.3200 - accuracy: 0.6860
Loss is 1.319950819015503,
Accuracy is 68.59999895095825


In [ ]:
### batch normalization epoch = 10, Loss is 1.2, acc=65.71
### original epoch = 15,bs = 32 Loss=0.85 , acc=70.89
###batch normalization epoch = 50,bs = 40 Loss=0.93 , acc=72.74
###batch normalization epoch = 100,bs = 32 Loss=0.84 , acc=74.29
###layer 4 acc=74,80
### layer 5 epoch = 25,bs = 64 Loss=1.31 , acc=68.59

In [23]:
model.save("l5e25b64.h5")

In [18]:
import tkinter as tk
from tkinter import filedialog
from tkinter import *
from PIL import ImageTk, Image
import numpy
#load the trained model to classify the images
from keras.models import load_model
model = load_model('bne100b32.h5')
#dictionary to label all the CIFAR-10 dataset classes.
classes = { 
    0:'Avião',
   1:'Carro',
   2:'Passaro',
   3:'Gato',
   4:'Cervo',
   5:'Cachorro',
   6:'Sapo',
   7:'Cavalo',
   8:'Navio',
   9:'Caminhão'
}
#initialise GUI
top=tk.Tk()
top.geometry('800x600')
top.title('Image Classification CIFAR10')
top.configure(background='#CDCDCD')
label=Label(top,background='#CDCDCD', font=('arial',15,'bold'))
sign_image = Label(top)
def classify(file_path):
    global label_packed
    image = Image.open(file_path)
    image = image.resize((32,32))
    image = numpy.expand_dims(image, axis=0)
    image = numpy.array(image)
    pred = np.argmax(model.predict([image])[0],axis=-1)
    sign = classes[pred]
#     print(sign)
    label.configure(foreground='#011638', text=sign) 
def show_classify_button(file_path):
    classify_b=Button(top,text="Classify Image",
   command=lambda: classify(file_path),padx=10,pady=5)
    classify_b.configure(background='#364156', foreground='white',
font=('arial',10,'bold'))
    classify_b.place(relx=0.79,rely=0.46)
def upload_image():
    try:
        file_path=filedialog.askopenfilename()
        uploaded=Image.open(file_path)
        uploaded.thumbnail(((top.winfo_width()/2.25),
        (top.winfo_height()/2.25)))
        im=ImageTk.PhotoImage(uploaded)
        sign_image.configure(image=im)
        sign_image.image=im
        label.configure(text='')
        show_classify_button(file_path)
    except:
        pass
upload=Button(top,text="Upload an image",command=upload_image,padx=10,pady=5)
upload.configure(background='#364156', foreground='white',font=('arial',10,'bold'))
upload.pack(side=BOTTOM,pady=50)
sign_image.pack(side=BOTTOM,expand=True)
label.pack(side=BOTTOM,expand=True)
heading = Label(top, text="Image Classification CIFAR10",pady=20, font=('arial',20,'bold'))
heading.configure(background='#CDCDCD',foreground='#364156')
heading.pack()
top.mainloop()